In [81]:
import pandas as pd
import random
from collections import Counter
import numpy as np
from sklearn.linear_model import LinearRegression
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [121]:
xcelLocation = "../xcels/"
# !export xcelLocation="../xcels/"
csvFileName = "MarketWatchPlus-1398_5_7"
csvFileName = "1382-2-2"
RECORD_THRESHOLD = 3000
TESTCASE_NUMBER = 20
alreadyPrep = False

In [122]:
!ls $xcelLocation | grep ".csv" > csvFiles
tmp = !cat csvFiles
csvFiles = [name[:-4] for name in tmp]

In [123]:
def readCsv(xcelLocation, csvFileName):
    df=pd.read_csv(xcelLocation + csvFileName + '.csv', sep=',',header=[0])
    return df

def addDiff(df): 
    diffTable = df.loc[:,["نام","کمترین","بیشترین"]]
    sumdif,count = (0,0)
    difs = []
    for row in range(len(diffTable)):
        minim = diffTable.loc[row][1]
        maxim = diffTable.loc[row][2]
        if(minim != 0):
            count += 1
            sumdif += (maxim - minim)/minim
            difs.append((maxim - minim)/minim * 100)
        else:
            difs.append(0)
    diffTable.loc[:,"diffs"] = difs
    df.loc[:,"diffs"] = difs
    return df, diffTable, sumdif, count

In [124]:
def prepare():
    global alreadyPrep
    global df
    global minMaxDF
    global testNames
    
    if alreadyPrep:
        return
    alreadyPrep = True
    
    df = readCsv(xcelLocation=xcelLocation, csvFileName="master")
    minMaxDF = df.loc[:,["نام","کمترین","بیشترین", "قیمت پایانی - مقدار", "year", "month", "day"]]
    minMaxDF.columns = ["name", "min", "max", "close", "year", "month", "day"]
    
    allNames = minMaxDF.name.tolist()
    names = sorted(list(set(minMaxDF.name))[1:])
    print(len(names))
    counts = Counter(allNames)
    testNames = []
    tmpNames = []
    for name in names:
        if counts[name] > RECORD_THRESHOLD:
            tmpNames.append(name)
    for i in range(TESTCASE_NUMBER):
        ran = random.randint(0, len(tmpNames))
        testNames.append(tmpNames[ran])
        tmpNames.remove(tmpNames[ran])
    print(len(testNames))

In [125]:
def runSimulator(testDF, minMaxPredictFunction):
    ans = testDF.iloc[len(testDF) - 1]
    testDF = testDF.drop(len(testDF) - 1)
    predictMin, predictMax = minMaxPredictFunction(testDF)

    result = 0
    if predictMin >= ans["min"] and predictMin <= ans["max"]:
        if predictMax <= ans["max"] and predictMax >= ans["min"]:
            result += 1
        else:
            result += 2
    return result

In [126]:
def calcRate(minMaxPredictFunction):
    prepare()
    SS = 0
    SF = 0
    FF = 0
    
    for name in testNames:
        testDF = minMaxDF.loc[minMaxDF["name"] == name].reset_index(drop=True)
        for i in range(200):
            result = runSimulator(testDF, minMaxPredictFunction)
            if result % 2 == 1:
                SS += 1
            if (result/2) % 2 == 1:
                SF += 1
            if result == 0:
                FF += 1
            testDF = testDF.drop([len(testDF) - 1, len(testDF) - 2, len(testDF) - 3, len(testDF) - 4])
    print(SS, " ", SF, " ", FF)
    print("prec ", SS/(SS + SF))
    print("rec ", (SS + SF)/(SS + FF + SF))
            

In [127]:
def rator1(inputDF):
    size = len(inputDF)
    return inputDF.iloc[size - 1]["min"]*102/100 , inputDF.iloc[size - 1]["max"]

In [128]:
def rator2(inputDF):
    regressor = Sequential()
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))

    regressor.add(Dense(units = 1))

    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

    regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [130]:
calcRate(rator1)

3128
20
888   787   2325
prec  0.5301492537313433
rec  0.41875
